# Data preparation
Cleaning of the data
Splitting the data in training/testing set
Create datasets per repository
Doing anything else that needs to be done

In [8]:
# Auxiliary
import logging as log
import pymongo

import pandas as pd
from pandas import json_normalize

name = './logs/03.log'
log.basicConfig(filename=name, level=log.DEBUG, force=True, format='%(asctime)s %(message)s: ', datefmt='%Y-%m-%d %H:%M:%S')

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']
coll_issues = db['01_Issues']

In [9]:
issues = coll_issues.find()
df = json_normalize(issues)

In [10]:
df

,_id,expand,id,self,key,repository,availability,originSprintId,totalChanges,sprintChanges,...,fields.parent.fields.issuetype.description,fields.parent.fields.issuetype.iconUrl,fields.parent.fields.issuetype.name,fields.parent.fields.issuetype.subtask,fields.parent.fields.issuetype.avatarId,fields.priority,fields.reporter,fields.assignee.emailAddress,fields.creator.emailAddress,fields.reporter.emailAddress
0,61d563dff4d395ee2213faea,"operations,versionedRepresentations,editmeta,c...",Jira467709,https://jira.atlassian.com/rest/api/2/issue/46...,SRCTREEWIN-3765,Jira,indirect,Jira2660,12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61d56651f4d395ee221413da,"operations,versionedRepresentations,editmeta,c...",Jira526575,https://jira.atlassian.com/rest/api/2/issue/52...,SRCTREE-3260,Jira,indirect,Jira2743,21,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61d56651f4d395ee221413de,"operations,versionedRepresentations,editmeta,c...",Jira518198,https://jira.atlassian.com/rest/api/2/issue/51...,SRCTREE-3255,Jira,indirect,Jira2743,8,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,61d56651f4d395ee2214140f,"operations,versionedRepresentations,editmeta,c...",Jira473042,https://jira.atlassian.com/rest/api/2/issue/47...,SRCTREE-3185,Jira,indirect,Jira2660,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61d56651f4d395ee22141411,"operations,versionedRepresentations,editmeta,c...",Jira472411,https://jira.atlassian.com/rest/api/2/issue/47...,SRCTREE-3182,Jira,indirect,Jira2660,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155437,61d62c36f4d395ee2226935e,"operations,versionedRepresentations,editmeta,c...",Apache13069994,https://issues.apache.org/jira/rest/api/2/issu...,ARIA-229,Apache,indirect,Apache336,9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155438,61d62c36f4d395ee222693cb,"operations,versionedRepresentations,editmeta,c...",Apache13049213,https://issues.apache.org/jira/rest/api/2/issu...,ARIA-118,Apache,indirect,Apache336,36,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155439,61d62c36f4d395ee222693f3,"operations,versionedRepresentations,editmeta,c...",Apache13037378,https://issues.apache.org/jira/rest/api/2/issu...,ARIA-75,Apache,indirect,Apache336,16,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155440,61d62c36f4d395ee22269426,"operations,versionedRepresentations,editmeta,c...",Apache13019966,https://issues.apache.org/jira/rest/api/2/issu...,ARIA-19,Apache,indirect,Apache329,16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for key in df.keys():
    print(key)

_id
expand
id
self
key
repository
availability
originSprintId
fields.fixVersions
fields.resolution
fields.lastViewed
fields.priority.self
fields.priority.iconUrl
fields.priority.name
fields.priority.id
fields.labels
fields.aggregatetimeoriginalestimate
fields.timeestimate
fields.versions
fields.issuelinks
fields.assignee.self
fields.assignee.name
fields.assignee.key
fields.assignee.avatarUrls.48x48
fields.assignee.avatarUrls.24x24
fields.assignee.avatarUrls.16x16
fields.assignee.avatarUrls.32x32
fields.assignee.displayName
fields.assignee.active
fields.assignee.timeZone
fields.status.self
fields.status.description
fields.status.iconUrl
fields.status.name
fields.status.id
fields.status.statusCategory.self
fields.status.statusCategory.id
fields.status.statusCategory.key
fields.status.statusCategory.colorName
fields.status.statusCategory.name
fields.components
fields.aggregatetimeestimate
fields.creator.self
fields.creator.name
fields.creator.key
fields.creator.avatarUrls.48x48
fields.cre